In [1]:
import numpy as np
import suncalc
import drawSvg as draw

# from numba import jit
from datetime import datetime

import random
# import pandas as pd

In [2]:
def base_date_arr(year):
    days = np.arange(str(year), str(year+1), dtype='datetime64[D]')

    # get minutes in each day as row vector
    lst = []
    for day in days:
        lst.append(np.arange(day, day+1, dtype='datetime64[m]'))

    # construct np.array
    lst_arr = np.array(lst)
    # transpose to get minutes as column vectors
    arr_dt64_noflip = lst_arr.T
    # flip array along axis 0 since SVG is filled from bottom left corner
    arr_dt64 = np.flip(arr_dt64_noflip, axis=0)
    
    return arr_dt64

# get days in a year
year = 2021
arr_dt64 = base_date_arr(year)

In [3]:
# convert from array of datetime64 to normal datetime with the UTC timestamp attached
def dt64_to_dtUTC_noVec(dt64):
    dt64 = np.datetime64(dt64)
    ts = (dt64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    return datetime.utcfromtimestamp(ts)

dt64_to_dtUTC = np.vectorize(dt64_to_dtUTC_noVec)

arr_dtUTC = dt64_to_dtUTC(arr_dt64)

<ipython-input-3-0fefeeef9e42>:4: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  ts = (dt64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')


In [4]:
# display(pd.DataFrame(arr_dtUTC))
# type(arr_dtUTC[0,0])

In [5]:
# display(pd.DataFrame(colored))

In [6]:
# for each timestamp, get sun azimuth and altitude
# then convert that into a color
def pos_noVec(ts, lon, lat):
    azi_alt = suncalc.get_position(ts, lon, lat)
    azi = azi_alt['azimuth']
    alt = azi_alt['altitude']
    # color = dan_color(azi, alt)
    return azi_alt

pos = np.vectorize(pos_noVec)

lon = 71.0589
lat = 42.3601

out = pos(arr_dtUTC, lon, lat)
# display(pd.DataFrame(out))

In [7]:
# arr_dtUTC.shape

In [8]:
# NOT IN USE
# TTD function to convert time into color
def time_to_color(time):
    color = "%06x" % random.randint(0, 0xFFFFFF)
    return color

vf = np.vectorize(time_to_color)

z = np.zeros((1440,365))

colored = vf(z)

In [9]:
# display(pd.DataFrame(colored))

In [10]:
width = 1920
height = 1080

x_tick = width / 365 # days in a year
y_tick = height / 1440 # min in a day

d = draw.Drawing(width, height, displayInline=False)

def gen_svg(colors, d, width, height, x_tick, y_tick):
    # x is the axis 1 index
    # y is the axis 0 index
    # index tuple is (y,x)
    
    it = np.nditer(colors, flags=['multi_index'])
    for color in it:
        y = it.multi_index[0] * y_tick
        x = it.multi_index[1] * x_tick
        r = draw.Rectangle(x, y, x_tick, y_tick, fill=f'#{color}')
        d.append(r)

gen_svg(colored, d, width, height, x_tick, y_tick)

d.setPixelScale(2)  # Set number of pixels per geometry unit

d.saveSvg('new.svg')
# d  # Display as SVG